In [2]:
import pandas as pd

# Load the CSV from your local path
file_path = r"C:\Users\lavki\OneDrive\Desktop\pandas\data_collection\data-collection\Wallet id - Sheet1.csv"
wallets_df = pd.read_csv(file_path)

# View the first few rows
print(wallets_df.head())

# Extract and clean wallet addresses
wallets = wallets_df.iloc[:, 0].dropna().str.lower().tolist()
print(f"✅ Total wallets loaded: {len(wallets)}")
print(wallets[:5])  # first 5 addresses


                                    wallet_id
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1  0x06b51c6882b27cb05e712185531c1f74996dd988
2  0x0795732aacc448030ef374374eaae57d2965c16c
3  0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4  0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
✅ Total wallets loaded: 103
['0x0039f22efb07a647557c7c5d17854cfd6d489ef3', '0x06b51c6882b27cb05e712185531c1f74996dd988', '0x0795732aacc448030ef374374eaae57d2965c16c', '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9', '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']


#**Step 2: Fetch Compound Protocol Data for Each Wallet
**

#**For each wallet, we want to fetch transactions like:

Supply (mint)

Borrow

Repay

Withdraw (redeem)

Liquidation events**#

#**Use Web3.py + The Graph (Subgraph) for Compound
More accurate on-chain data

Slower and needs more setup (infura or alchemy node + GraphQL)**#

#**Define GraphQL endpoint

Create functions to query:

Borrow

Repay

Supply (Mint)

Redeem

Liquidation

Loop through wallet list

Save per-wallet data into a structured format (CSV or DataFrame)**#

In [4]:
import requests
import pandas as pd
import time

# Graph endpoint
GRAPH_API_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"


In [5]:
def get_borrow_events(wallet_address):
    query = """
    {
      borrowEvents(
        where: {borrower: "%s"}, 
        first: 1000, 
        orderBy: blockTimestamp, 
        orderDirection: desc
      ) {
        id
        amount
        reserve
        blockTimestamp
        txHash
      }
    }
    """ % wallet_address.lower()

    try:
        response = requests.post(GRAPH_API_URL, json={'query': query})
        data = response.json()
        return data['data']['borrowEvents']
    except Exception as e:
        print(f"Error for wallet {wallet_address}: {e}")
        return []


In [7]:
import pandas as pd

# Load wallet CSV from local machine
file_path = r"C:\Users\lavki\OneDrive\Desktop\pandas\data_collection\data-collection\Wallet id - Sheet1.csv"
wallets_df = pd.read_csv(file_path)

# Check the first few rows
wallets_df.head()


,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae


In [8]:
# Extract unique wallet addresses from the first column
wallets = wallets_df.iloc[:, 0].dropna().unique().tolist()

# Show count and first 5 wallets
print(f"Total wallets loaded: {len(wallets)}")
print(wallets[:5])


Total wallets loaded: 103
['0x0039f22efb07a647557c7c5d17854cfd6d489ef3', '0x06b51c6882b27cb05e712185531c1f74996dd988', '0x0795732aacc448030ef374374eaae57d2965c16c', '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9', '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']


In [9]:
import time

GRAPH_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

def run_query(query):
    response = requests.post(GRAPH_URL, json={'query': query})
    while response.status_code != 200:
        time.sleep(1)
        response = requests.post(GRAPH_URL, json={'query': query})
    return response.json()

def get_events(wallet, event_type):
    all_data = []
    skip = 0
    while True:
        query = f"""
        {{
          {event_type}s(first: 1000, skip: {skip}, where: {{ borrower: "{wallet.lower()}" }}) {{
            amount
            timestamp
          }}
        }}
        """
        data = run_query(query).get("data", {}).get(f"{event_type}s", [])
        if not data:
            break
        all_data.extend(data)
        skip += 1000
    return all_data


In [12]:
from tqdm import tqdm

wallet_features = []

for wallet in tqdm(wallets):
  # Limit for now to 10 wallets for testing
    borrows = get_events(wallet, "borrow")
    repays = get_events(wallet, "repay")
    liquidations = get_events(wallet, "liquidation")

    borrow_total = sum(float(tx['amount']) for tx in borrows)
    repay_total = sum(float(tx['amount']) for tx in repays)
    liquidation_count = len(liquidations)

    all_timestamps = [int(tx['timestamp']) for tx in borrows + repays + liquidations]
    active_days = (max(all_timestamps) - min(all_timestamps)) // (60 * 60 * 24) if all_timestamps else 0

    wallet_features.append({
        "wallet": wallet,
        "total_borrowed": borrow_total,
        "total_repaid": repay_total,
        "num_borrows": len(borrows),
        "num_repays": len(repays),
        "num_liquidations": liquidation_count,
        "active_days": active_days
    })


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [00:47<00:00,  2.16it/s]


In [13]:
import pandas as pd

df_features = pd.DataFrame(wallet_features)
df_features.head()


,wallet,total_borrowed,total_repaid,num_borrows,num_repays,num_liquidations,active_days
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,0,0,0,0,0,0
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0,0,0,0,0,0
2,0x0795732aacc448030ef374374eaae57d2965c16c,0,0,0,0,0,0
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,0,0,0,0,0,0
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,0,0,0,0,0,0


In [17]:
%whos


Variable            Type         Data/Info
------------------------------------------
GRAPH_API_URL       str          https://api.thegraph.com/<...>graphprotocol/compound-v2
GRAPH_URL           str          https://api.thegraph.com/<...>graphprotocol/compound-v2
active_days         int          0
all_timestamps      list         n=0
borrow_total        int          0
borrows             list         n=0
df_features         DataFrame                             <...>n\n[103 rows x 7 columns]
file_path           str          C:\Users\lavki\OneDrive\D<...>on\Wallet id - Sheet1.csv
get_borrow_events   function     <function get_borrow_even<...>ts at 0x000001D1C1DB0E00>
get_events          function     <function get_events at 0x000001D1C1F76840>
liquidation_count   int          0
liquidations        list         n=0
pd                  module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
repay_total         int          0
repays              list         n=0
requests       

In [18]:
active_wallets_df = df_features[
    (df_features["total_borrowed"] > 0) |
    (df_features["total_repaid"] > 0) |
    (df_features["num_borrows"] > 0) |
    (df_features["num_repays"] > 0) |
    (df_features["num_liquidations"] > 0)
]

active_wallets_df.head()


,wallet,total_borrowed,total_repaid,num_borrows,num_repays,num_liquidations,active_days


In [19]:
import numpy as np

def compute_risk_score(row):
    # Handle division safely
    repayment_ratio = row['total_repaid'] / row['total_borrowed'] if row['total_borrowed'] > 0 else 1
    liquidation_penalty = row['num_liquidations'] * 50
    repayment_penalty = (1 - repayment_ratio) * 300  # more weight to repayment
    active_days_score = min(row['active_days'], 100)  # cap influence of very long activity
    
    # Start from a base score
    risk = 1000 - (repayment_penalty + liquidation_penalty) + active_days_score
    return int(np.clip(risk, 0, 1000))  # Ensure between 0 and 1000


In [20]:
df_features["score"] = df_features.apply(compute_risk_score, axis=1)
df_features[["wallet", "score"]].head()


,wallet,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,1000
1,0x06b51c6882b27cb05e712185531c1f74996dd988,1000
2,0x0795732aacc448030ef374374eaae57d2965c16c,1000
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,1000
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,1000
